In [1]:
import pandas as pd
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary

import pyLDAvis.gensim_models
import pyLDAvis.sklearn
#pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from gensim.models import CoherenceModel

In [2]:
def compute_coherence_values(dictionary, 
                             corpus, 
                             texts, 
                             max_topics, 
                             min_topics, 
                             step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    
    for num_topics in range(min_topics, max_topics, step):

        model = LdaModel(corpus = corpus, 
                         num_topics = num_topics, 
                         id2word = dictionary,
                         update_every = 1,
                         passes = 10,
                         per_word_topics = True)

        model_list.append(model)
        coherencemodel = CoherenceModel(model = model, 
                                        texts = texts, 
                                        dictionary = dictionary, 
                                        coherence='c_v')

        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [3]:
df1 = pd.read_pickle('prepared_txt/all_titles_da_df.pkl')

In [4]:
bigram = gensim.models.Phrases(df1['lemmatized_titles'])
titles_bigrams = [bigram[line] for line in df1['lemmatized_titles']]

In [5]:
dictionary = Dictionary(titles_bigrams)
corpus = [dictionary.doc2bow(text) for text in titles_bigrams]

In [16]:
min_topics = 2
max_topics = 10
step = 2
x = range(min_topics, max_topics, step)

In [17]:
model_list, coherence_values = compute_coherence_values(dictionary = dictionary, 
                                                        corpus = corpus, 
                                                        texts = titles_bigrams, 
                                                        min_topics = min_topics, 
                                                        max_topics = max_topics, 
                                                        step = step)

In [18]:
best_result_index = coherence_values.index(max(coherence_values))
print(x[best_result_index])


#optimal_model = model_list[best_result_index]
# Select the model and print the topics
#model_topics = optimal_model.show_topics(formatted=False)
#print(f'''The {x[best_result_index]} topic gives the highest coherence score \\
#of {coherence_values[best_result_index]}''')

2


In [19]:
ldamodel = LdaModel(corpus = corpus, 
                    num_topics = x[best_result_index], 
                    id2word = dictionary,
                    update_every = 1,
                    passes = 10,
                    per_word_topics = True)

In [10]:
#pyLDAvis.enable_notebook()
model =pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
#pyLDAvis.display(model)
pyLDAvis.save_html(model, 'models/da/all_titles_da.html')

/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the

In [ ]:
#import matplotlib
#import matplotlib.pyplot as plt

#min_topics = 1
#max_topics= 30
#step = 1
#x = range(min_topics, max_topics, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')